In [1]:
import git2vectors
from chat_utils import RetrievalChain

c:\Users\voyno\Desktop\code\repos\repo-chat\venv\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Create vectorstore, this will take a while
repo = "https://github.com/smol-ai/developer"
git2vectors.create_vectorstore(repo)

In [2]:
# load vectorstore, this is fast
vectorstore = git2vectors.get_vectorstore()

In [3]:
# Create an instance of the class
chain = RetrievalChain(
    git2vectors.OPENAI_API_KEY, vectorstore)

# Let's say we have a query
query = "How do I use this?"

# generic retrieval query
response = chain.chat(query)

print(response['query'])
print(response['text'])
print(response['similar_documents'])
print(response['scores'])
print(response['callback'])

How do I use this?
Inadequate context. Please provide more information about the code repository and what you are trying to accomplish with it.
['modal-client\r\nopenai\r\ntiktoken', 'body {\r\n  font-family: Arial, sans-serif;\r\n  margin: 0;\r\n  padding: 15px;\r\n}\r\n\r\ntextarea, input {\r\n  display: block;\r\n  width: 100%;\r\n  margin-bottom: 10px;\r\n  padding: 5px;\r\n  font-size: 14px;\r\n}\r\n\r\nbutton {\r\n  background-color: #4CAF50;\r\n  border: none;\r\n  color: white;\r\n  padding: 10px 20px;\r\n  text-align: center;\r\n  text-decoration: none;\r\n  display: inline-block;\r\n  font-size: 14px;\r\n  margin: 10px 0;\r\n  cursor: pointer;\r\n}\r\n\r\nbutton:hover {\r\n  background-color: #45a049;\r\n}\r\n\r\n#content {\r\n  margin-top: 20px;\r\n  font-size: 14px;\r\n  line-height: 1.5;\r\n}\r\n\r\n#loadingIndicator {\r\n  display: none;\r\n  font-size: 14px;\r\n  color: #999;\r\n}', '<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n  <meta nam

In [4]:
# Create an instance of the class (with upgrade query)
chain = RetrievalChain(
    git2vectors.OPENAI_API_KEY, vectorstore, upgrade=True)

# Let's say we have a query
query = "How do I use this?"

# retrieval query with query upgrade augmentation
response = chain.chat(query)

print(response['query'])
print(response['text'])
print(response['similar_documents'])
print(response['scores'])
print(response['callback'])

Can you provide instructions on how to use this code repository?
Inadequate context. The documents provided do not contain specific instructions on how to use a code repository. Can you please provide more information about the repository in question?
["- simply add to the prompt as they discover underspecified parts of the prompt\r\n  - manually runs the code and identifies errors\r\n  - *paste the error into the prompt* just like they would file a github issue\r\n  - for extra help, they can use `debugger.py` which reads the whole codebase to make specific code change suggestions\r\n\r\nLoop until happiness is attained. Notice that AI is only used as long as it is adding value - once it gets in your way, just take over the codebase from your smol junior developer with no fuss and no hurt feelings. (*we could also have smol-dev take over an existing codebase and bootstrap its own prompt... but that's a Future Direction*)\r\n\r\n*Not no code, not low code, but some third thing.* \r\n\r